# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[ 4.674985  ,  8.213466  , -9.075721  ,  9.568374  ,  8.454808  ,
        -1.2327975 ,  3.3903713 , -7.828242  , -0.8454461 ,  0.6288572 ],
       [-4.243999  ,  5.6107078 , -5.669777  , -1.7957245 , -9.255528  ,
         0.71774375,  4.4435897 , -2.8747153 , -5.090096  ,  9.684121  ],
       [ 5.2615476 , -4.0487256 ,  4.464928  , -2.9367518 ,  3.5061095 ,
        -4.016832  , -3.4638855 ,  6.078449  , -6.953326  , -1.004144  ],
       [-3.008261  ,  4.625961  , -4.483249  ,  2.228457  ,  1.643532  ,
        -2.4505196 , -5.258201  , -1.6679403 , -7.985754  ,  2.8311467 ],
       [-5.607241  ,  2.2695985 , -3.7516537 , -1.8182003 , -5.143028  ,
         7.599363  ,  2.8252366 ,  8.773042  ,  1.6198314 ,  1.1772048 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

2022-10-17 18:31:15,013 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-17 18:31:15,013 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-17 18:31:15,018 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-10-17 18:31:15,018 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 45.78 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34409,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 45.78 GiB
Comm: tcp://127.0.0.1:45563,Total threads: 1
Dashboard: http://127.0.0.1:39877/status,Memory: 22.89 GiB
Nanny: tcp://127.0.0.1:34535,


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

2022-10-17 18:31:20,904 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-825a1a92-c781-43b1-a07b-3b74f674f288
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-10-17 18:31:20,923 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-8dcbdfd3-8404-4583-b738-40a300477444
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_display_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_display_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-10-17 18:31:20,945 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-0c28f46a-596d-4c96-bf1c-41dc4e439c07
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception

2022-10-17 18:31:21,119 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-921d1d36-0105-467b-85b3-5e8067680745
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_png_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_png_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-10-17 18:31:21,146 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-4c4a0269-19d4-4351-9c3f-ea362db5342d
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-10-17 18:31:21,169 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-a1fdd84f-6b5b-49e5-b7e7-ed23c34ccf1b
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_pdf_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_pdf_ does

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[-2.8722029 ,  4.469733  , -4.431363  ,  2.3996615 ,  1.7438415 ,
        -2.4938557 , -5.2212667 , -1.7067922 , -8.130271  ,  2.640922  ,
        -4.307933  ,  5.579306  , -5.741948  , -1.7193332 , -9.359336  ,
         0.7162488 ,  4.4438014 , -2.9173872 , -4.9321446 ,  9.692951  ,
         8.393692  , -6.2387223 , -6.363846  ,  1.963377  ,  4.162584  ,
        -9.159683  ,  4.6117425 ,  8.8011265 ,  6.855182  ,  2.2458146 ],
       [ 4.7991467 ,  8.40242   , -9.21459   ,  9.392472  ,  8.512869  ,
        -1.0980053 ,  3.3258235 , -7.802849  , -0.5990244 ,  0.25806773,
         5.5174656 , -4.1132016 ,  4.29229   , -2.8411753 ,  3.6327322 ,
        -4.173101  , -3.6205482 ,  6.2173705 , -6.9105287 , -1.084521  ,
        -5.8539176 ,  2.2375815 , -3.8543427 , -1.6783282 , -5.3225746 ,
         7.5756173 ,  2.9321432 ,  8.521326  ,  1.5875132 ,  1.0917971 ],
       [-4.655767  , -9.605046  ,  6.6638994 ,  4.435602  ,  2.156688  ,
         2.599673  ,  0.6010148 ,  6.262877  , -8

## Exporting cuML Random Forest models for inferencing on machines without GPUs

Starting with cuML version 21.06, you can export cuML Random Forest models and run predictions with them on machines without an NVIDIA GPUs. The [Treelite](https://github.com/dmlc/treelite) package defines an efficient exchange format that lets you portably move the cuML Random Forest models to other machines. We will refer to the exchange format as "checkpoints."

Here are the steps to export the model:

1. Call `to_treelite_checkpoint()` to obtain the checkpoint file from the cuML Random Forest model.

In [14]:
from cuml.ensemble import RandomForestClassifier as cumlRandomForestClassifier
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
X, y = X.astype(np.float32), y.astype(np.int32)
clf = cumlRandomForestClassifier(max_depth=3, random_state=0, n_estimators=10)
clf.fit(X, y)

checkpoint_path = './checkpoint.tl'
# Export cuML RF model as Treelite checkpoint
clf.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

/opt/conda/envs/rapids/lib/python3.9/site-packages/cuml/internals/api_decorators.py:794: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


2. Copy the generated checkpoint file `checkpoint.tl` to another machine on which you'd like to run predictions.

3. On the target machine, install Treelite by running `pip install treelite` or `conda install -c conda-forge treelite`. The machine does not need to have an NVIDIA GPUs and does not need to have cuML installed.

4. You can now load the model from the checkpoint, by running the following on the target machine:

In [15]:
import treelite

# The checkpoint file has been copied over
checkpoint_path = './checkpoint.tl'
tl_model = treelite.Model.deserialize(checkpoint_path)
out_prob = treelite.gtil.predict(tl_model, X, pred_margin=True)
print(out_prob)

[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.  